# Segmenting and Clustering Neighborhoods in Toronto - Third Part

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

### Libraries importing

In [2]:
pip install geocoder

     |████████████████████████████████| 102kB 7.6MB/s ta 0:00:011
     |████████████████████████████████| 92kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 829kB 17.2MB/s eta 0:00:01     |███████████████████             | 491kB 17.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

## 1. Data load and reducing

Let's first load Toronto's data, including its boroughs coordinates.

For illustration purposes, let's simplify Toronto's map and segment and cluster only the neighborhoods containing the word "Toronto" in their names. So let's slice the original dataframe and create a new dataframe with this data.

In [5]:
coord_Toronto = pd.read_csv('coord_Toronto.csv') 

In [6]:
coord_Toronto.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern , Rouge",43.808626,-79.189913
1,1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
2,2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.765806,-79.185284
3,3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [7]:
coord_Toronto = coord_Toronto.drop('Unnamed: 0', 1)
coord_Toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [8]:
reduced_Toronto = coord_Toronto[coord_Toronto['Borough'].str.contains('Toronto')]
reduced_Toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,EastToronto,The Beaches,43.678148,-79.295349
41,M4K,EastToronto,"The Danforth West , Riverdale",43.683424,-79.354564
42,M4L,EastToronto,"India Bazaar , The Beaches West",43.668291,-79.315578
43,M4M,EastToronto,Studio District,43.648000,-79.339260
44,M4N,CentralToronto,Lawrence Park,43.729455,-79.386415


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(reduced_Toronto['Borough'].unique()),
        reduced_Toronto.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


Let's visualize the neighborhoods. First, we need Toronto's city coordinates.

In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
# create map of Toronto using latitude and longitude values
reduced_map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(reduced_Toronto['Latitude'], reduced_Toronto['Longitude'], reduced_Toronto['Borough'], reduced_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        #parse_html=False).
        #.add_to(map_toronto)  
        parse_html=False).add_to(reduced_map_toronto)  
    
reduced_map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'UT00WXQB1DSZBAEGF3EEVXKREOSWEKNDZDUSOAQ1BF044JPY' # your Foursquare ID
CLIENT_SECRET = 'J2FI0CCKM0KS5WC12SGOCTMZANIIOA2BAR2D2XVLAOSWPYUW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UT00WXQB1DSZBAEGF3EEVXKREOSWEKNDZDUSOAQ1BF044JPY
CLIENT_SECRET:J2FI0CCKM0KS5WC12SGOCTMZANIIOA2BAR2D2XVLAOSWPYUW


### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name. First, we reset the index:

In [13]:
reduced_Toronto = reduced_Toronto.reset_index()
reduced_Toronto.head()

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,EastToronto,The Beaches,43.678148,-79.295349
1,41,M4K,EastToronto,"The Danforth West , Riverdale",43.683424,-79.354564
2,42,M4L,EastToronto,"India Bazaar , The Beaches West",43.668291,-79.315578
3,43,M4M,EastToronto,Studio District,43.648000,-79.339260
4,44,M4N,CentralToronto,Lawrence Park,43.729455,-79.386415


In [14]:
reduced_Toronto = reduced_Toronto.drop('index', 1)
reduced_Toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,EastToronto,The Beaches,43.678148,-79.295349
1,M4K,EastToronto,"The Danforth West , Riverdale",43.683424,-79.354564
2,M4L,EastToronto,"India Bazaar , The Beaches West",43.668291,-79.315578
3,M4M,EastToronto,Studio District,43.648000,-79.339260
4,M4N,CentralToronto,Lawrence Park,43.729455,-79.386415


In [15]:
reduced_Toronto.loc[0, 'Neighbourhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [16]:
neighborhood_latitude = reduced_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = reduced_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = reduced_Toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67814827600006, -79.29534930999995.


### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format( CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

Send the GET request and examine the resutls

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea5d690edbcad001b168913'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.68264828050006,
    'lng': -79.28913883913675},
   'sw': {'lat': 43.673648271500056, 'lng': -79.30155978086314}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Calvary Baptist Church,Church,43.681059,-79.299246


And how many venues were returned by Foursquare?

In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Manhattan

### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [24]:
toronto_venues = getNearbyVenues(names=reduced_Toronto['Neighbourhood'],
                                   latitudes=reduced_Toronto['Latitude'],
                                   longitudes=reduced_Toronto['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

### Let's check the size of the resulting dataframe

In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(1595, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.678148,-79.295349,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.678148,-79.295349,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.678148,-79.295349,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.678148,-79.295349,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.678148,-79.295349,Calvary Baptist Church,43.681059,-79.299246,Church


Let's check how many venues were returned for each neighborhood

In [27]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,68,68,68,68,68,68
"Brockton , Parkdale Village , Exhibition Place",45,45,45,45,45,45
Business reply mail Processing CentrE,100,100,100,100,100,100
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",65,65,65,65,65,65
Central Bay Street,52,52,52,52,52,52
Christie,12,12,12,12,12,12
Church and Wellesley,80,80,80,80,80,80
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,29,29,29,29,29,29


### Let's find out how many unique categories can be curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 219 uniques categories.


# 3. Analyze Each Neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [31]:
toronto_onehot.shape

(1595, 219)

In [32]:
toronto_onehot.to_csv('onehot_Toronto.csv')

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.014706,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.014706,0.029412,0.000000,0.000000,0.0,0.014706,0.000000,0.029412,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.029412,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.00,0.014706,0.044118,0.088235,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.014706,0.000000,0.014706,0.014706,0.000000,0.014706,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.00,0.000000,0.000000,0.029412,0.014706,0.000000,0.014706,0.000000,0.000000,0.000000,0.00,0.014706,0.014706,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0

#### Let's confirm the new size

In [34]:
toronto_grouped.shape

(38, 219)

#### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3         Cheese Shop  0.03
4              Lounge  0.03


----Brockton , Parkdale Village , Exhibition Place----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2        Diner  0.04
3  Pizza Place  0.04
4    Gift Shop  0.04


----Business reply mail Processing CentrE----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2  Japanese Restaurant  0.04
3                 Café  0.04
4  American Restaurant  0.03


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
               venue  freq
0        Coffee Shop  0.08
1               Café  0.06
2         Restaurant  0.06
3               Park  0.05
4  French Restaurant  0.05


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.19
1                      

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Hotel,Farmers Market,Italian Restaurant,Bakery,Restaurant,Beer Bar,Lounge
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Café,Pizza Place,Gift Shop,Thrift / Vintage Store,Diner,Boutique,Italian Restaurant,Brewery,Japanese Restaurant
2,Business reply mail Processing CentrE,Coffee Shop,Hotel,Café,Japanese Restaurant,American Restaurant,Restaurant,Asian Restaurant,Sandwich Place,Sushi Restaurant,Steakhouse
3,"CN Tower , King and Spadina , Railway Lands , ...",Coffee Shop,Café,Restaurant,Park,French Restaurant,Bar,Speakeasy,Lounge,Italian Restaurant,Gym / Fitness Center
4,Central Bay Street,Coffee Shop,Restaurant,Café,Sandwich Place,Middle Eastern Restaurant,Bubble Tea Shop,Breakfast Spot,Plaza,Clothing Store,Italian Restaurant


In [40]:
neighborhoods_venues_sorted.to_csv('neighborhoods_venues_sorted_Toronto.csv')

## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [43]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [51]:
toronto_merged = reduced_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,EastToronto,The Beaches,43.678148,-79.295349,0.0,Health Food Store,Trail,Pub,Church,Cupcake Shop,Dance Studio,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,M4K,EastToronto,"The Danforth West , Riverdale",43.683424,-79.354564,0.0,Bus Line,Business Service,Grocery Store,Park,Discount Store,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,M4L,EastToronto,"India Bazaar , The Beaches West",43.668291,-79.315578,0.0,Fast Food Restaurant,Gym,Italian Restaurant,Fish & Chips Shop,Pet Store,Liquor Store,Steakhouse,Sushi Restaurant,Sandwich Place,Movie Theater
3,M4M,EastToronto,Studio District,43.648000,-79.339260,0.0,Baseball Field,Business Service,Athletics & Sports,Government Building,Night Market,Women's Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,M4N,CentralToronto,Lawrence Park,43.729455,-79.386415,2.0,Bus Line,Swim School,Women's Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Finally, let's visualize the resulting clusters

In [59]:
toronto_merged.dropna()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,EastToronto,The Beaches,43.678148,-79.295349,0.0,Health Food Store,Trail,Pub,Church,Cupcake Shop,Dance Studio,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,M4K,EastToronto,"The Danforth West , Riverdale",43.683424,-79.354564,0.0,Bus Line,Business Service,Grocery Store,Park,Discount Store,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,M4L,EastToronto,"India Bazaar , The Beaches West",43.668291,-79.315578,0.0,Fast Food Restaurant,Gym,Italian Restaurant,Fish & Chips Shop,Pet Store,Liquor Store,Steakhouse,Sushi Restaurant,Sandwich Place,Movie Theater
3,M4M,EastToronto,Studio District,43.648000,-79.339260,0.0,Baseball Field,Business Service,Athletics & Sports,Government Building,Night Market,Women's Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,M4N,CentralToronto,Lawrence Park,43.729455,-79.386415,2.0,Bus Line,Swim School,Women's Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,M4P,CentralToronto,Davisville North,43.713171,-79.388870,0.0,Food & Drink Shop,Breakfast Spot,Gym,Department Store,Park,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
6,M4R,CentralToronto,North Toronto West,43.714139,-79.406456,1.0,Playground,Gym Pool,Park,Garden,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
7,M4S,CentralToronto,Davisville,43.703327,-79.385649,0.0,Dessert Shop,Coffee Shop,Café,Sandwich Place,Pizza Place,Italian Restaurant,Flower Shop,Fast Food Restaurant,Farmers Market,Diner
8,M4T,CentralToronto,"Moore Park , Summerhill East",43.690328,-79.383522,1.0,Playground,Gym,Trail,Park,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
9,M4V,CentralToronto,"Summerhill West , Rathnelly , South Hill , For...",43.686378,-79.402372,0.0,Athletics & Sports,Light Rail Station,Coffee Shop,Skating Rink,Café,Liquor Store,Supermarket,Women's Store,Electronics Store,Fish Market


In [62]:
toronto_merged.dropna(axis = 1, how = 'all')

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,EastToronto,The Beaches,43.678148,-79.295349,0.0,Health Food Store,Trail,Pub,Church,Cupcake Shop,Dance Studio,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,M4K,EastToronto,"The Danforth West , Riverdale",43.683424,-79.354564,0.0,Bus Line,Business Service,Grocery Store,Park,Discount Store,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,M4L,EastToronto,"India Bazaar , The Beaches West",43.668291,-79.315578,0.0,Fast Food Restaurant,Gym,Italian Restaurant,Fish & Chips Shop,Pet Store,Liquor Store,Steakhouse,Sushi Restaurant,Sandwich Place,Movie Theater
3,M4M,EastToronto,Studio District,43.648000,-79.339260,0.0,Baseball Field,Business Service,Athletics & Sports,Government Building,Night Market,Women's Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,M4N,CentralToronto,Lawrence Park,43.729455,-79.386415,2.0,Bus Line,Swim School,Women's Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,M4P,CentralToronto,Davisville North,43.713171,-79.388870,0.0,Food & Drink Shop,Breakfast Spot,Gym,Department Store,Park,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
6,M4R,CentralToronto,North Toronto West,43.714139,-79.406456,1.0,Playground,Gym Pool,Park,Garden,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
7,M4S,CentralToronto,Davisville,43.703327,-79.385649,0.0,Dessert Shop,Coffee Shop,Café,Sandwich Place,Pizza Place,Italian Restaurant,Flower Shop,Fast Food Restaurant,Farmers Market,Diner
8,M4T,CentralToronto,"Moore Park , Summerhill East",43.690328,-79.383522,1.0,Playground,Gym,Trail,Park,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
9,M4V,CentralToronto,"Summerhill West , Rathnelly , South Hill , For...",43.686378,-79.402372,0.0,Athletics & Sports,Light Rail Station,Coffee Shop,Skating Rink,Café,Liquor Store,Supermarket,Women's Store,Electronics Store,Fish Market


In [66]:
toronto_merged = toronto_merged.fillna(0)

In [65]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EastToronto,0,Health Food Store,Trail,Pub,Church,Cupcake Shop,Dance Studio,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,EastToronto,0,Bus Line,Business Service,Grocery Store,Park,Discount Store,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,EastToronto,0,Fast Food Restaurant,Gym,Italian Restaurant,Fish & Chips Shop,Pet Store,Liquor Store,Steakhouse,Sushi Restaurant,Sandwich Place,Movie Theater
3,EastToronto,0,Baseball Field,Business Service,Athletics & Sports,Government Building,Night Market,Women's Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,CentralToronto,0,Food & Drink Shop,Breakfast Spot,Gym,Department Store,Park,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
7,CentralToronto,0,Dessert Shop,Coffee Shop,Café,Sandwich Place,Pizza Place,Italian Restaurant,Flower Shop,Fast Food Restaurant,Farmers Market,Diner
9,CentralToronto,0,Athletics & Sports,Light Rail Station,Coffee Shop,Skating Rink,Café,Liquor Store,Supermarket,Women's Store,Electronics Store,Fish Market
11,DowntownToronto,0,Coffee Shop,Bakery,Café,Italian Restaurant,Restaurant,Pub,Market,Pizza Place,Deli / Bodega,Caribbean Restaurant
12,DowntownToronto,0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Pub,Café,Dance Studio,Mediterranean Restaurant,Men's Store
13,DowntownToronto,0,Pub,Café,Coffee Shop,Athletics & Sports,Performing Arts Venue,Tech Startup,Seafood Restaurant,Mexican Restaurant,Food Truck,French Restaurant


#### Cluster 2

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,CentralToronto,1,Playground,Gym Pool,Park,Garden,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
8,CentralToronto,1,Playground,Gym,Trail,Park,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
10,DowntownToronto,1,Playground,Grocery Store,Park,Candy Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 3

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,CentralToronto,2,Bus Line,Swim School,Women's Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 4

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,DowntownToronto,3,Harbor / Marina,Farm,Park,Theme Park,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


#### Cluster 5

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,CentralToronto,4,Gym / Fitness Center,Restaurant,Women's Store,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
